In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
original_df = pd.read_csv("weather_data.csv")


C:\Users\nhk06\AppData\Local\Temp\ipykernel_30492\3155273763.py:1: DtypeWarning: Columns (1,2,3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv("weather_data.csv")


In [3]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82536 entries, 0 to 82535
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TIME UTC                      82536 non-null  object 
 1   Temp Definition °C            82287 non-null  object 
 2   Dew Point Definition °C       82288 non-null  object 
 3   Rel Hum Definition %          82288 non-null  object 
 4   Precip. Amount Definition mm  82288 non-null  object 
 5   Wind Dir Definition 10's deg  38548 non-null  object 
 6   Wind Spd Definition km/h      38548 non-null  object 
 7   Visibility Definition km      0 non-null      float64
 8   Stn Press Definition kPa      82288 non-null  object 
 9   Hmdx Definition               13528 non-null  float64
 10  Wind Chill Definition         0 non-null      float64
 11  Weather Definition            82288 non-null  object 
dtypes: float64(3), object(9)
memory usage: 7.6+ MB


In [25]:
original_df.drop(columns=["Wind Dir Definition 10's deg", "Wind Spd Definition km/h", "Visibility Definition km", \
                          "Wind Chill Definition", "Hmdx Definition", "Weather Definition"], inplace=True)

In [27]:
original_df.to_csv("weather_data_drop.csv", index=False)

#### Drop: <i>Wind Dir Definition 10's deg, Wind Spd Definition km/h, Visibility Definition km, Wind Chill Definition, Hmdx Definition, Weather Definition.</i>

Gắn thêm cột ngày, tháng, năm vào để dễ làm việc

In [3]:
original_df.drop(columns=["Wind Dir Definition 10's deg", "Wind Spd Definition km/h", "Visibility Definition km", \
                          "Wind Chill Definition", "Hmdx Definition", "Weather Definition"], inplace=True)
original_df['Time'] = pd.to_datetime(original_df['TIME UTC']).dt.time
original_df['Day'] = pd.to_datetime(original_df['TIME UTC']).dt.day # get the day of Time UTC
original_df['Month'] = pd.to_datetime(original_df['TIME UTC']).dt.month
original_df['Year'] = pd.to_datetime(original_df['TIME UTC']).dt.year

#### Handle missing value: 
- Không thể drop missing value do đây là dữ liệu có yếu tố thời gian.

- Chiến lược
    -> Điền vào các cột trống giá trị của hàng cùng giờ, cùng ngày, cùng tháng nhưng có giá trị năm trong quá khứ gần nhất.
        => Cùng tháng, cùng mùa trong năm thì trong phần lớn trường hợp sẽ tương đồng nhau về nhiệt độ, độ ẩm và các yếu tố khác


In [4]:
import pandas as pd

def fill_missing(column: str, df: pd.DataFrame):
    NaN_indices = df[pd.isna(df[column])].index
    
    for i in NaN_indices:
        year = df.at[i, 'Year']
        month = df.at[i, 'Month']
        day = df.at[i, 'Day']
        time = df.at[i, 'Time']
        filled = False
        
        # Tìm giá trị trong quá khứ
        for j in range(year - 1, 2015 - 1, -1):
            value = df[(df['Time'] == time) & (df['Day'] == day) & (df['Month'] == month) & (df['Year'] == j)]
            if not value.empty and not pd.isna(value[column].iloc[0]):
                df.at[i, column] = (value[column].iloc[0])
                filled = True
                break
        
        # Tìm giá trị trong tương lai
        if not filled:
            for j in range(year + 1, 2024 + 1):
                value = df[(df['Time'] == time) & (df['Day'] == day) & (df['Month'] == month) & (df['Year'] == j)]
                if not value.empty and not pd.isna(value[column].iloc[0]):
                    df.at[i, column] = (value[column].iloc[0])
                    filled = True
                    break

columns2Fill = ['Temp Definition °C', 'Dew Point Definition °C', 'Rel Hum Definition %', 'Precip. Amount Definition mm', 'Stn Press Definition kPa']

for column in columns2Fill:
    fill_missing(column, original_df)

In [15]:
original_df.drop(columns=['Year', 'Month', 'Day','Time']).info(verbose=True, memory_usage=False, )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82536 entries, 0 to 82535
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   TIME UTC                      82536 non-null  object
 1   Temp Definition °C            82536 non-null  object
 2   Dew Point Definition °C       82536 non-null  object
 3   Rel Hum Definition %          82536 non-null  object
 4   Precip. Amount Definition mm  82536 non-null  object
 5   Stn Press Definition kPa      82536 non-null  object
dtypes: object(6)

In [6]:
for column in columns2Fill:
    original_df[column] = pd.to_numeric(original_df[column], errors='coerce')

In [7]:
original_df.to_csv('handledMising.csv')

In [8]:
# Ensure the columns are numeric and handle errors by coercing them to NaN
numeric_columns = ['Temp Definition °C', 'Dew Point Definition °C', 'Rel Hum Definition %', \
                    'Precip. Amount Definition mm', 'Stn Press Definition kPa']

for column in numeric_columns:
    original_df[column] = pd.to_numeric(original_df[column], errors='coerce')

# Nhóm dữ liệu theo ngày, tháng, năm
grouped_df = original_df.groupby(['Year', 'Month', 'Day'])

# Tính giá trị trung bình, lớn nhất và nhỏ nhất cho các thuộc tính
aggregated_df = grouped_df.agg({
    'Temp Definition °C': ['mean', 'max', 'min'],
    'Dew Point Definition °C': ['mean', 'max', 'min'],
    'Rel Hum Definition %': ['mean', 'max', 'min'],
    'Precip. Amount Definition mm': ['sum'],
    'Stn Press Definition kPa': ['mean', 'max', 'min']
}).reset_index()

# Đổi tên các cột cho dễ đọc
aggregated_df.columns = ['Year', 'Month', 'Day',
                         'Temp_mean', 'Temp_max', 'Temp_min',
                         'DewPoint_mean', 'DewPoint_max', 'DewPoint_min',
                         'RelHum_mean', 'RelHum_max', 'RelHum_min',
                         'PrecipAmount_sum',
                         'StnPress_mean', 'StnPress_max', 'StnPress_min']

new_numberic_columns = ['Temp_mean', 'Temp_max', 'Temp_min',
                         'DewPoint_mean', 'DewPoint_max', 'DewPoint_min',
                         'RelHum_mean', 'RelHum_max', 'RelHum_min',
                         'PrecipAmount_sum',
                         'StnPress_mean', 'StnPress_max', 'StnPress_min']

for column in new_numberic_columns:
    aggregated_df[column] = aggregated_df[column].round(3)
    

# Lưu DataFrame đã được tính toán vào file CSV
aggregated_df.to_csv('preProcessedData.csv', index=False)

aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3439 entries, 0 to 3438
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              3439 non-null   int32  
 1   Month             3439 non-null   int32  
 2   Day               3439 non-null   int32  
 3   Temp_mean         3439 non-null   float64
 4   Temp_max          3439 non-null   float64
 5   Temp_min          3439 non-null   float64
 6   DewPoint_mean     3439 non-null   float64
 7   DewPoint_max      3439 non-null   float64
 8   DewPoint_min      3439 non-null   float64
 9   RelHum_mean       3439 non-null   float64
 10  RelHum_max        3439 non-null   float64
 11  RelHum_min        3439 non-null   float64
 12  PrecipAmount_sum  3439 non-null   float64
 13  StnPress_mean     3439 non-null   float64
 14  StnPress_max      3439 non-null   float64
 15  StnPress_min      3439 non-null   float64
dtypes: float64(13), int32(3)
memory usage: 389

In [10]:
import pandas as pd

# Đọc dữ liệu đã được tiền xử lý
df = pd.read_csv('preProcessedData.csv')

In [11]:
df.drop(columns=['Year', 'Month', 'Day']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3439 entries, 0 to 3438
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temp_mean         3439 non-null   float64
 1   Temp_max          3439 non-null   float64
 2   Temp_min          3439 non-null   float64
 3   DewPoint_mean     3439 non-null   float64
 4   DewPoint_max      3439 non-null   float64
 5   DewPoint_min      3439 non-null   float64
 6   RelHum_mean       3439 non-null   float64
 7   RelHum_max        3439 non-null   float64
 8   RelHum_min        3439 non-null   float64
 9   PrecipAmount_sum  3439 non-null   float64
 10  StnPress_mean     3439 non-null   float64
 11  StnPress_max      3439 non-null   float64
 12  StnPress_min      3439 non-null   float64
dtypes: float64(13)
memory usage: 349.4 KB
